# Segmenting and Clustering Neighborhoods in Toronto
## Peer-graded Assignment:  to explore and cluster the neighborhoods in Toronto
### *by Leopoldo Sprandel*

### Importing data from wikipedia

In [140]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests as rq

In [141]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page= rq.get(wikipedia_link).text

In [142]:
raw_list = page[page.find('wikitable sortable')-15:page.find('/table')]
raw_list = pd.read_html(raw_list, header=0, index_col=None, attrs={"class":"wikitable sortable"})[0]
raw_list.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Wrangling data
Ignore cells with a borough that is Not assigned  
Two rows with the same name (on Neighbourhood) will be combined into one row with the neighborhoods separated with a comma 
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough  
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [154]:
postals=raw_list
# Rename the column Postcode by PostalCode
postals.rename(columns={'Postcode':'PostalCode'}, inplace=True)
# Check the columns
postals.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [156]:
#check if missing data
missing_data = postals.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

PostalCode
False    289
Name: PostalCode, dtype: int64

Borough
False    289
Name: Borough, dtype: int64

Neighbourhood
False    289
Name: Neighbourhood, dtype: int64



In [ ]:
# Drop the 
postals.dropna(subset=["Borough"], axis=0, inplace = True)